# Capstone Project - The Battle of the Neighborhoods (Week 1)

## Introduction <a name="introduction"></a>

### The Problem
In this project we will try to find optimal location to open a Chinese restaurant. This report will be targeted to stakeholders interested in open such restaurant in **Toronto**, Canada.

We will try to detect **locations that are not already crowded with Chinese restaurant**. We would also prefer locations **as close to downtown as possible**, assuming that the first condition met.

### Data <a name="data"></a>
Based on definition of our problem, factors that will influence our decision are:
* number of existing Chinese restaurants
* distance of neighborhood from downtown

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be found from [Wiki page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) and this latitude and longitude [File](http://cocl.us/Geospatial_data)
* number of Chinese restaurants in every neighborhood will be obtained using **Foursquare API**
* coordinate of Toronto downtown has been found through a simple Google search (43.6548, -79.3883)

#### Neighborhood Candidates
First, get neighborhood information from the [Wiki page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). The table contains Postcode, Borough, and Neighbourhood information. We will use the existing Postcode area as our area candidates, and there are 103 such candidates. 
We could find Toronto Postcode geographic coordinates from an online [file](http://cocl.us/Geospatial_data). We will combine the 2 tables into 1 dataframe that contains Postcode, Neighborhood, Latitude, and Longitude.

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!pip install lxml

# scraping table from webpage
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)

# cleaning 
tables = table[0]
df = pd.DataFrame(tables)
df = df[df.Borough != 'Not assigned']
df = df.replace("Not assigned", "Queen's Park")
df = df.groupby('Postcode').agg({'Borough':'min','Neighbourhood':', '.join})
df = df.reset_index()

# read the latitude and longitude csv file into a dataframe
url='http://cocl.us/Geospatial_data'
ll = pd.read_csv(url)

# add latitude and longitude info into the dataframe
new = pd.concat([df,ll], axis=1)
new = new.drop(['Postal Code'], axis=1)
new.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### Foursquare
Then, we will use Foursquare to get Chinese restaurant venues information in each area.

In [2]:
# Foursquare Info
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [3]:
# Import needed libraries
import requests 
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

In [4]:
# Function to get all the venues
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category', 'Venue Id']
    
    return(nearby_venues)

In [5]:
# Call function to get all venues
Venues = getNearbyVenues(names=new['Postcode'], 
                         latitudes=new['Latitude'], 
                         longitudes=new['Longitude'])
Venues

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


,Area,Area Latitude,Area Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Venue Id
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,4bf58dd8d48988d16e941735
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,4bf58dd8d48988d116941735
2,M1C,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target,4f2a23984b9023bd5841ed2c
3,M1C,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum,4bf58dd8d48988d190941735
4,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,4bf58dd8d48988d122951735
5,M1E,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa,4bf58dd8d48988d1ed941735
6,M1E,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant,4bf58dd8d48988d1c1941735
7,M1E,43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location,4bf58dd8d48988d1ef941735
8,M1E,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank,4bf58dd8d48988d10a951735
9,M1E,43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center,4bf58dd8d48988d104941735


Now, let's filter for all the Chinese restaurants:

In [22]:
column_names = ['Area', 'Area Latitude', 'Area Longitude', 'Venue Name', 
                'Venue Latitude', 'Venue Longitude', 'Venue Category', 'Venue Id']
chinese_restaurants = pd.DataFrame(columns=column_names)

# Category IDs corresponding to Chinese restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):
chinese_restaurant_categories = ['4bf58dd8d48988d145941735', '52af3a5e3cf9994f4e043bea', '52af3a7c3cf9994f4e043bed', 
                                 '58daa1558bbb0b01f18ec1d3', '52af3a673cf9994f4e043beb', '52af3a903cf9994f4e043bee', 
                                 '4bf58dd8d48988d1f5931735', '52af3a9f3cf9994f4e043bef', '52af3aaa3cf9994f4e043bf0', 
                                 '52af3ab53cf9994f4e043bf1', '52af3abe3cf9994f4e043bf2', '52af3ac83cf9994f4e043bf3', 
                                 '52af3ad23cf9994f4e043bf4', '52af3add3cf9994f4e043bf5', '52af3af23cf9994f4e043bf7', 
                                 '52af3ae63cf9994f4e043bf6', '52af3afc3cf9994f4e043bf8', '52af3b053cf9994f4e043bf9', 
                                 '52af3b213cf9994f4e043bfa', '52af3b293cf9994f4e043bfb', '52af3b343cf9994f4e043bfc', 
                                 '52af3b3b3cf9994f4e043bfd', '52af3b463cf9994f4e043bfe', '52af3b633cf9994f4e043c01', 
                                 '52af3b513cf9994f4e043bff', '52af3b593cf9994f4e043c00', '52af3b6e3cf9994f4e043c02', 
                                 '52af3b773cf9994f4e043c03', '52af3b813cf9994f4e043c04', '52af3b893cf9994f4e043c05', 
                                 '52af3b913cf9994f4e043c06', '52af3b9a3cf9994f4e043c07', '52af3ba23cf9994f4e043c08']

ID_list = Venues['Venue Id'].tolist()

for ids in ID_list:
    if ids in chinese_restaurant_categories:
        row = Venues.loc[Venues['Venue Id'] == ids]
        chinese_restaurants = chinese_restaurants.append(row, ignore_index = True) 
chinese_restaurants

,Area,Area Latitude,Area Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Venue Id
0,M1H,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant,52af3ac83cf9994f4e043bf3
1,M1P,43.757410,-79.273304,Kim Kim restaurant,43.753833,-79.276611,Chinese Restaurant,4bf58dd8d48988d145941735
2,M1T,43.781638,-79.304302,The Royal Chinese Restaurant 避風塘小炒,43.780505,-79.298844,Chinese Restaurant,4bf58dd8d48988d145941735
3,M1W,43.799525,-79.318389,Mr Congee Chinese Cuisine 龍粥記,43.798879,-79.318335,Chinese Restaurant,4bf58dd8d48988d145941735
4,M1W,43.799525,-79.318389,Phoenix Restaurant 金鳳餐廳,43.798198,-79.318432,Chinese Restaurant,4bf58dd8d48988d145941735
5,M2J,43.778517,-79.346556,Manchu Wok,43.778225,-79.343302,Chinese Restaurant,4bf58dd8d48988d145941735
6,M2K,43.786947,-79.385975,Sun Star Chinese Cuisine 翠景小炒,43.787914,-79.381234,Chinese Restaurant,4bf58dd8d48988d145941735
7,M3C,43.725900,-79.340923,Congee Star 帝王名粥,43.726586,-79.341833,Chinese Restaurant,4bf58dd8d48988d145941735
8,M3H,43.754328,-79.442259,China Court,43.755780,-79.437437,Chinese Restaurant,4bf58dd8d48988d145941735
9,M4R,43.715383,-79.405678,C'est Bon,43.716785,-79.400406,Chinese Restaurant,4bf58dd8d48988d145941735


We found all the Chinese restaurant in Toronto area, and we are ready for analyzing.